# Averaged Artifact Substraction with EEG Data
## First step prepare Jupyter Notebook

In [ ]:
import os
from FACET.Facet import Facet
from loguru import logger
import sys
os.getcwd()
os.chdir(os.path.dirname(os.path.abspath("__file__")))

eegDataSet_filepath = "NiazyFMRI.edf"
motiondata_moosmann_filepath = "headmotiondata.tsv"
export_bids_path = "./bids_dir"

#logging configuration
logger.remove()
logger.add(sys.stderr, level="INFO")
logger.add("Facet.log", level="DEBUG")

#defining important parameters
window_size = 25
upsampling_factor = 10
relative_trigger_position = -0.01
relative_window_position = 0.5
moosmann_motion_threshold = 0.8
event_id_description_pairs={'trigger':1}
regex_trigger_annotation_filter = r'\btrigger\b' # Annotations with the description 'trigger' are considered as triggers
regex_trigger_event_filter = r'\b1\b' # Events with the value 1 are considered as triggers
unwanted_bad_channels = ['EKG', 'EMG', 'EOG', 'ECG'] # Channels with these names are considered as bad channels and not considered in the processing
evaluation_measures=["SNR", "RMS", "RMS2", "MEDIAN"]

f = Facet()


## Second Step import EEG Data

In [ ]:
f.import_EEG(eegDataSet_filepath, rel_trig_pos=relative_trigger_position, upsampling_factor=upsampling_factor, bads=unwanted_bad_channels)
f.find_triggers(regex_trigger_event_filter)
eeg = f.get_EEG()

f.export_as_bids(event_id_description_pairs)
f.import_from_bids(rel_trig_pos=relative_trigger_position, bads=unwanted_bad_channels)
eeg = f.get_EEG()

f.plot_EEG(title="after import")

## Third Step Preprocessing

In [ ]:
f.pre_processing()


## Fourth Step find triggers

In [ ]:

f.find_triggers(regex_trigger_annotation_filter, idx=0) # Using Niazys data
f.get_analytics().print_analytics()
eeg = f.get_EEG()


## Fourth step apply AAS

In [ ]:
f.apply_AAS(window_size=window_size, rel_window_position=relative_window_position)

## Fifth step Remove Artifacts

In [ ]:
f.remove_artifacts( )
eeg = f.get_EEG()



## Sixth step Post-Processing

In [ ]:
f.post_processing()

## Seventh display the processed eeg data

In [ ]:
f.plot_EEG(title="after lowpass")

## Eighth Evaluate Results

In [ ]:
f.find_triggers(r'\btrigger\b') # Using Niazys data
eeg_python = f.get_eeg()
f.add_to_evaluate(eeg_python, name="numpy_matrix") # Matrix will result in >1.0 RMS2 because of python mne inaccuracies

#apply moosmann
f.import_EEG(eegDataSet_filepath, rel_trig_pos=relative_trigger_position, bads=unwanted_bad_channels)
f.pre_processing()
f.find_triggers(regex_trigger_event_filter)
f.apply_Moosmann(file_path=motiondata_moosmann_filepath, threshold=moosmann_motion_threshold)
f.remove_artifacts()
f.post_processing()
f.find_triggers(regex_trigger_event_filter)
f.add_to_evaluate(f.get_eeg(), name="Moosmann")
results = f.evaluate(plot=True,measures=evaluation_measures)
logger.info(results)

f.export_as_bids(event_id=event_id_description_pairs, bids_path=export_bids_path)

